# Geothermal Datathon 2021
---

<img src="../figures/the_roaring_kitties_logo.png" width="200" align="center">

Team members:
- **Ricardo Lara**
- **Artur Davletshin**
- **Aigul Akberova**
- **Sercan Gul**
- **Hakki Aydin**
- **Jose Hernandez**


---

### Import Modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

### Define functions

In [191]:
def interpolate_syn_temp(df_original, df_synthetic):
    df = df_original.copy(deep=True)
    df_syn = df_synthetic.copy(deep=True)

    df['Synthetic Temp, C'] = np.NAN

    UWIs = df_syn.UWI.unique()
    
    for i, UWI in enumerate(UWIs):
        try:
            x = df.loc[df.UWI == UWI, 'Depth subsea, m'][0]
        except KeyError:
            continue

        depths = df_syn.loc[df_syn.UWI == UWI,'Depth subsea, m'].to_list()
        temps = df_syn.loc[df_syn.UWI == UWI,'Synthetic Temp, C'].to_list()

        if x < depths[-1]:
            x1 = max([d for d in depths if d < x])
            x2 = min([d for d in depths if d > x])
            t1 = temps[depths.index(x1)]
            t2 = temps[depths.index(x2)]
        else:
            x1 = depths[-2]
            x2 = depths[-1]
            t1 = temps[depths.index(x1)]
            t2 = temps[depths.index(x2)]
            
        t = t1 + (x - x1) * (t2 - t1) / (x2 - x1)
            
        df.loc[df.UWI == UWI, 'Synthetic Temp, C'] = t
    
    return df

def get_static_temp(df_original, df_static):
    df = df_original.copy(deep=True)
    df2 = df_static.copy(deep=True)

    df3 = df.merge(df2, on='UWI', how='left')

    df3['static_diff_depth'] = df3['Depth subsea, m_y'] - df3['Depth subsea, m_x']

    df4 = df.merge(df3[['UWI', 'Static Temp, C']], on='UWI', how='left')

    return df4


def get_synthetic_temp(df_original, df_synthetic):
    df = df_original.copy(deep=True)
    df2 = df_synthetic.copy(deep=True)

    df3 = df.merge(df2, on='UWI', how='left')

    df3['syn_diff_depth'] = df3['Depth subsea, m_y'] - df3['Depth subsea, m_x']
    df3['diff_depth_abs'] = np.abs(df3['syn_diff_depth'])

    idx = df3.groupby(['UWI'])['diff_depth_abs'].transform(min) == df3['diff_depth_abs']

    truetemp = df3.loc[idx, ['UWI', 'Synthetic Temp, C']]

    df4 = df.merge(truetemp, on='UWI', how='left')

    return df4


# 1 Eaglebine

## 1.1 Load temperature data

In [32]:
EB = pd.read_excel('../data/Eaglebine/Eaglebine BHT TSC data for SPE April 21 2020.xlsx') # Bottom-hole temperatures
EB_syn = pd.read_excel('../data/Eaglebine/Eaglebine TrueTemp_Train2.xlsx') # Synthetic temperatures
EB_TL = pd.read_csv('../data/Data_static_logs.csv') # Temperature static logs
assign = pd.read_csv("../data/set_assign.csv") # labels for train and validation sets

UWIs_test = assign.loc[assign.Set == 'Validation_Testing', 'UWI']
UWIs_train = assign.loc[assign.Set == 'Training', 'UWI']

### Convert unis to SI

In [33]:
# Eaglebine BHT
columns = ['TD (ft)', 'GL(ft)', 'BHT_below sea level (ft)', 'BHT_ subsurface (ft)']
EB[['TD, m', 'GL, m', 'Depth subsea, m', 'Depth subsurface, m',]]  = EB[columns] / 3.28084
EB['BHT, C'] = (EB['BHTorMRT (maximum recorded temperature) oF']-32)*5/9

# Eaglebine True
EB_syn['Depth subsea, m'] = EB_syn['Depth sub-sea (feet)'] / 3.28084
EB_syn['True Temperature, C'] = (EB_syn['True Temperature   (oF)'] - 32) * 5 / 9

# Log temps
EB_TL['Depth, m'] = TL['Depth (ft)'] / 3.28084

### Rename columns

In [31]:
EB.columns.to_list()

['UWI',
 'SurfLat',
 'SurfLong',
 'TD, ft',
 'GL, ft',
 'Depth subsea, ft',
 'Depth subsurface, ft',
 'BHT, F',
 'TSC, h',
 'TD, m',
 'GL, m',
 'Depth subsea, m',
 'Depth subsurface, m',
 'BHT, C']

In [29]:
# Rename columns
col_names = ['UWI',
                'SurfLat',
                'SurfLong',
                'TD, ft',
                'GL, ft',
                'Depth subsea, ft',
                'Depth subsurface, ft',
                'BHT, F',
                'TSC, h', 
                'TD, m',
                'GL, m',
                'Depth subsea, m',
                'Depth subsurface, m',
                'BHT, C'
                ]

EB.columns = col_names

# Select subset of features in SI units
EB2 = EB[['UWI',
        'SurfLat',
        'SurfLong',
        'TD, m',
        'GL, m',
        'Depth subsea, m',
        'Depth subsurface, m',
        'TSC, h',
        'BHT, C'
]]

In [34]:
EB_syn.columns.to_list()

['UWI',
 'Depth sub-sea (feet)',
 'True Temperature   (oF)',
 'Depth subsea, m',
 'True Temperature, C']

In [43]:
col_names = ['UWI',
            'Depth subsea, ft',
            'Synthetic Temp, F',
            'Depth subsea, m',
            'Synthetic Temp, C']

EB_syn.columns = col_names

EB_syn2 = EB_syn[['UWI',
                'Depth subsea, m',
                'Synthetic Temp, C']]

In [36]:
EB_TL.columns.to_list()

['Well_ID', 'Depth (ft)', 'Temp (degC)', 'Field', 'Depth, m']

In [41]:
col_names = ['UWI',
            'Depth, ft',
            'Static Temp, C',
            'Field',
            'Depth, m',
            ]

EB_TL.columns = col_names

# Select subset of features in SI units
EB_TL2 = EB_TL[['UWI',
        'Depth, m',
        'Static Temp, C',
        'Field'
        ]]

In [64]:
EB2.UWI = EB2.UWI.astype('str');
EB_syn2.UWI = EB_syn2.UWI.astype('str');
EB_TL2.UWI = EB_TL2.UWI.astype('str');
assign.UWI = assign.UWI.astype('str');

## 1.2 Add Synthetic and Static Temperatures

In [65]:
EB3 = interpolate_syn_temp(EB2, EB_syn2)
EB3.head()

,UWI,SurfLat,SurfLong,"TD, m","GL, m","Depth subsea, m","Depth subsurface, m","TSC, h","BHT, C","Synthetic Temp, C"
0,42013301410000,28.690426,-98.470138,5982.919009,105.430317,3725.417881,3830.848197,5.50,61.666667,141.773056
1,42013301930000,28.779921,-98.208313,3284.219895,121.127516,3168.700699,3289.828215,11.00,107.222222,132.005556
2,42013302760000,28.994360,-98.421799,1880.006340,140.329916,1747.418344,1887.748260,5.50,65.555556,88.974111
3,42013305480000,28.759118,-98.156410,2498.750320,125.089916,2379.268724,2504.358640,5.00,76.666667,108.009111
4,42013310190000,28.817400,-98.155319,3352.799893,155.118811,3203.289401,3358.408213,7.75,98.888889,133.797009


In [68]:
EB4 = EB3.merge(TL2[['UWI','Static Temp, C']], on='UWI', how='left')
EB4.head()

,UWI,SurfLat,SurfLong,"TD, m","GL, m","Depth subsea, m","Depth subsurface, m","TSC, h","BHT, C","Synthetic Temp, C","Static Temp, C"
0,42013301410000,28.690426,-98.470138,5982.919009,105.430317,3725.417881,3830.848197,5.50,61.666667,141.773056,NaN
1,42013301930000,28.779921,-98.208313,3284.219895,121.127516,3168.700699,3289.828215,11.00,107.222222,132.005556,132.778
2,42013302760000,28.994360,-98.421799,1880.006340,140.329916,1747.418344,1887.748260,5.50,65.555556,88.974111,NaN
3,42013305480000,28.759118,-98.156410,2498.750320,125.089916,2379.268724,2504.358640,5.00,76.666667,108.009111,NaN
4,42013310190000,28.817400,-98.155319,3352.799893,155.118811,3203.289401,3358.408213,7.75,98.888889,133.797009,NaN


## 1.3 Merge Synthetic and Static Temperatures


In [73]:
EB4["True Temp, C"] =  EB4["Static Temp, C"].fillna(EB4['Synthetic Temp, C'])
EB4["source"] = np.NAN
EB4.loc[EB4['Synthetic Temp, C'].notnull(), 'source'] = 'synthetic'
EB4.loc[EB4['Static Temp, C'].notnull(), 'source'] = 'static'
EB4.head(10)

,UWI,SurfLat,SurfLong,"TD, m","GL, m","Depth subsea, m","Depth subsurface, m","TSC, h","BHT, C","Synthetic Temp, C","Static Temp, C","True Temp, C",source
0,42013301410000,28.690426,-98.470138,5982.919009,105.430317,3725.417881,3830.848197,5.50,61.666667,141.773056,NaN,141.773056,synthetic
1,42013301930000,28.779921,-98.208313,3284.219895,121.127516,3168.700699,3289.828215,11.00,107.222222,132.005556,132.778,132.778000,static
2,42013302760000,28.994360,-98.421799,1880.006340,140.329916,1747.418344,1887.748260,5.50,65.555556,88.974111,NaN,88.974111,synthetic
3,42013305480000,28.759118,-98.156410,2498.750320,125.089916,2379.268724,2504.358640,5.00,76.666667,108.009111,NaN,108.009111,synthetic
4,42013310190000,28.817400,-98.155319,3352.799893,155.118811,3203.289401,3358.408213,7.75,98.888889,133.797009,NaN,133.797009,synthetic
5,42013311740000,28.819071,-98.151352,3339.388693,160.913059,3151.775155,3312.688214,9.00,118.333333,132.491486,NaN,132.491486,synthetic
6,42013312950000,28.927915,-98.577467,2022.957535,158.922715,1879.701540,2038.624255,3.00,62.777778,93.000111,NaN,93.000111,synthetic
7,42013313140000,28.694047,-98.765861,2612.745516,130.881116,2484.729520,2615.610636,8.00,85.000000,111.688222,NaN,111.688222,synthetic
8,42013313780000,28.851271,-98.771820,2221.382329,157.322515,1880.692140,2038.014655,7.00,65.000000,93.658472,NaN,93.658472,synthetic
9,42013325590000,28.987249,-98.701927,1610.258348,170.200315,626.363980,796.564295,1.00,37.777778,49.954444,NaN,49.954444,synthetic


In [78]:
EB4['label'] = 'not assigned'
EB4.loc[EB4.UWI.isin(UWIs_test), 'label'] = 'test'
EB4.loc[EB4.UWI.isin(UWIs_train), 'label'] = 'train'
EB4.head()

,UWI,SurfLat,SurfLong,"TD, m","GL, m","Depth subsea, m","Depth subsurface, m","TSC, h","BHT, C","Synthetic Temp, C","Static Temp, C","True Temp, C",source,label
0,42013301410000,28.690426,-98.470138,5982.919009,105.430317,3725.417881,3830.848197,5.50,61.666667,141.773056,NaN,141.773056,synthetic,train
1,42013301930000,28.779921,-98.208313,3284.219895,121.127516,3168.700699,3289.828215,11.00,107.222222,132.005556,132.778,132.778000,static,train
2,42013302760000,28.994360,-98.421799,1880.006340,140.329916,1747.418344,1887.748260,5.50,65.555556,88.974111,NaN,88.974111,synthetic,train
3,42013305480000,28.759118,-98.156410,2498.750320,125.089916,2379.268724,2504.358640,5.00,76.666667,108.009111,NaN,108.009111,synthetic,train
4,42013310190000,28.817400,-98.155319,3352.799893,155.118811,3203.289401,3358.408213,7.75,98.888889,133.797009,NaN,133.797009,synthetic,train


In [79]:
EB4.to_csv('tidy data/Eaglebine_Temperatures.csv', index=False)

# 2 Duvernay

## 2.1 Load temperature data

In [180]:
DV = pd.read_excel('../data/Duvernay/Duvernay DST BHT for SPE April 20 2021.xlsx')
DV_syn = pd.read_excel('../data/Duvernay/Duvenay TrueTemp_Train.xlsx')
DV_TL = pd.read_csv('../data/Data_static_logs.csv')

In [181]:
DV_TL['Depth, m'] = DV_TL['Depth (ft)'] / 3.28084

#* add extra column 
DV['Depth subsea, m']= - (DV['elevation M above sea level'] - (DV['DST Start Depth (MD) (m)'] + DV['DST End Depth (MD) (m)'])/2)


In [182]:
columns = ['UWI',
        'Depth subsea, m',
        'DST Bottom Hole Temp. (degC)',
        'Formation DSTd',
]

DV2 = DV[columns]

col_names = ['UWI',
                'Depth subsea, m',
                'BHT, C',
                'Formation'
                ]

DV2.columns = col_names

In [183]:
column_names = ['UWI',
                'Depth subsea, m',
                'Synthetic Temp, C',
                ]

DV_syn.columns = column_names

In [184]:

column_names = ['UWI',
                'Depth subsea, ft',
                'Static Temp, C',
                'Field',
                'Depth subsea, m']

DV_TL.columns = column_names



In [186]:
DV2.UWI = DV2.UWI.astype('str');
DV_syn.UWI = DV_syn.UWI.astype('str');
DV_TL.UWI = DV_TL.UWI.astype('str');

## 2.2 Add Synthetic and Static Temperatures

In [192]:
DV3 = get_synthetic_temp(DV2, DV_syn)
DV3.head()

,UWI,"Depth subsea, m","BHT, C",Formation,"Synthetic Temp, C"
0,100010107020W500,716.0,48.33,TRmontney,60.97
1,100010608109W500,1044.4,34.26,Dmuskeg,63.98
2,100010904012W500,2424.2,104.40,Melkton,NaN
3,100011206021W400,171.9,29.00,Dwintrbrn,39.07
4,100011506915W500,936.4,64.00,Dwabamun,69.47


In [193]:
DV4 = get_static_temp(DV3, DV_TL)
DV4.head()

,UWI,"Depth subsea, m","BHT, C",Formation,"Synthetic Temp, C","Static Temp, C"
0,100010107020W500,716.0,48.33,TRmontney,60.97,NaN
1,100010608109W500,1044.4,34.26,Dmuskeg,63.98,NaN
2,100010904012W500,2424.2,104.40,Melkton,NaN,NaN
3,100011206021W400,171.9,29.00,Dwintrbrn,39.07,NaN
4,100011506915W500,936.4,64.00,Dwabamun,69.47,NaN


In [194]:
DV4["True Temp, C"] =  DV4["Static Temp, C"].fillna(DV4['Synthetic Temp, C'])
DV4["source"] = np.NAN
DV4.loc[~DV4['Synthetic Temp, C'].isnull(), 'source'] = 'synthetic'
DV4.loc[~DV4['Static Temp, C'].isnull(), 'source'] = 'static'

In [196]:
DV4['label'] = 'not assigned'
DV4.loc[DV4.UWI.isin(UWIs_test), 'label'] = 'test'
DV4.loc[DV4.UWI.isin(UWIs_train), 'label'] = 'train'

In [198]:
DV4.to_csv('tidy data/Duvernay_Temperatures.csv', index=False)